In [3]:
import numpy as np
import warnings

In [9]:
def write_revision(file_name, n_addr):    
    revision_string = f"""
    -------------------------------------------------------------------------
    -- File   : {file_name}
    -------------------------------------------------------------------------
    -- Description : Synchronous {n_addr} x 1 ROM.
    -------------------------------------------------------------------------
    -- Revision:
    --     Date        Rev     Author             Description
    --     12/07/2023  1.0     Henrique L. Silva  Creation.
  	--     20/07/2023  2.0     Antonio V. S. Neto Update on 'memory_struct'
	  --                                            to std_logic_vector to
	  --                                            force Quartus to infer ROM.
    -------------------------------------------------------------------------"""

    return revision_string

def write_entity(entity_name, n_bits_addr):
    entity_string = f"""
    library ieee;
    use ieee.std_logic_1164.all;
    use ieee.numeric_std.all;

    entity {entity_name} is
    port (
            address   : in  std_logic_vector({n_bits_addr-1} downto 0);
            clk       : in  std_logic;
            out_rom   : out std_logic
        );
    end entity {entity_name};
    """

    return entity_string

def write_architecture_head(entity_name, n_addr):  
    architecture_head_string = f"""
    -- Initial data explicitly declared
    architecture {entity_name}_arch of {entity_name} is
      type memory_struct is array(0 to {n_addr-1}) of std_logic_vector(0 downto 0);
    """
    return architecture_head_string

def write_memory(rom_values):
  memory_string = """
    signal memory : memory_struct := ({});
  """

  # The format command casts the values from integers to binaries.
  # The binary number must have wordSize bits, so zeros are padded to the left.
  # The join function will skip lines, put the commas and tab the code correctly.

  memory_string = memory_string.format(',\n \t \t \t \t'.join([f'"{format(x)}"' for x in rom_values]), format(0))

  return memory_string
  
def write_architecture_body(entity_name):
  architecture_body_string = f"""
    begin
    -- Reads ROM position at every rising edge of 'clk'.
    process (clk)
    begin
    if (clk'event and clk = '1') then
        out_rom <= memory(to_integer(unsigned(address)))(0);
      end if;
    end process;

  end architecture {entity_name}_arch;  
  """
  
  return architecture_body_string

In [10]:
def write_rom(file_name, entity_name, n_addr, rom_values):
    '''
    This generates a ROM implemented in VHDL.

    Parameters
    ----------------
    file_name: string
        The name of the VHDL file that will be generated.
    
    entity_name: string
        ROM's entity name.

    n_addr: int
        The number of address that the ROM will have. 
    
    wordSize: int
        The size of the word that the ROM will store.
    
    rom_values: np.array
        The values that will be written into the ROM.
    '''

    # Number of bits-1 needed to reach the number of addresses
    
    n_bits_addr = np.int8(np.ceil(np.log2(n_addr)))

    # Verify if n_addr is divisible by 2^n. If not, append zeros to the values until you get a multiple.
    if not np.log2(n_addr).is_integer():
        while rom_values.shape[0] < 2**n_bits_addr:
            rom_values = np.append(rom_values, 0)
            n_addr = 2**n_bits_addr
        warnings.warn("The number of addresses it not an exponent of 2. You may have addresses with value output to zero.")
    
    # Verify if the lengths match, if not append zeros to the end
    if rom_values.shape[0] < n_addr:
        while rom_values.shape[0] < n_addr:
            rom_values = np.append(rom_values, 0)
        warnings.warn("You have more addresses than values. The list will be appended with zeros to match addresses' length.")

    revision_string = write_revision(file_name, n_addr)
    entity_string = write_entity(entity_name, n_bits_addr)
    architecture_head_string = write_architecture_head(entity_name, n_addr)
    memory_string = write_memory(rom_values)
    architecture_body_string = write_architecture_body(entity_name)

    # Concatenate the strings
    VHDL = revision_string + entity_string + architecture_head_string + memory_string + architecture_body_string

    # Write the file
    fout = open(f'{file_name}.vhd', 'w')

    # Write the string into the file
    fout.write(VHDL)

    # Close the file
    fout.close()

    # Return the complete string
    return VHDL

In [11]:
# Arquivo Exemplo

# Final VHDL file name
file_name = "stimulus_list_bit_example" 

# ROM Entity name
entity_name = file_name

# ROM Entity name
n_addr = 8

# The values that will be written
rom_values = np.random.randint(0, 2, n_addr)

# Write the ROM
result = write_rom(file_name, entity_name, n_addr, rom_values)

# Print result
print(result)


    -------------------------------------------------------------------------
    -- File   : stimulus_list_bit_example
    -------------------------------------------------------------------------
    -- Description : Synchronous 8 x 1 ROM.
    -------------------------------------------------------------------------
    -- Revision:
    --     Date        Rev     Author             Description
    --     12/07/2023  1.0     Henrique L. Silva  Creation.
  	--     20/07/2023  2.0     Antonio V. S. Neto Update on 'memory_struct'
	  --                                            to std_logic_vector to
	  --                                            force Quartus to infer ROM.
    -------------------------------------------------------------------------
    library ieee;
    use ieee.std_logic_1164.all;
    use ieee.numeric_std.all;

    entity stimulus_list_bit_example is
    port (
            address   : in  std_logic_vector(2 downto 0);
            clk       : in  std_logic;
        

<h2>Write ROM for clock enable stimuli</h2>

In [12]:
############## ENABLE RF 1 ##############
# Final VHDL file name
file_name = "stimulus_list_clk_enable_rf_1" 

# ROM Entity name
entity_name = file_name

# ROM Entity name
n_addr = 94

# Creating the array (using Antonio suggestion)
rom_values = []
rom_values.append(0)

for i in range(60):
    rom_values.append(1)

for i in range(4):
    rom_values.append(0)
    rom_values.append(0)
    rom_values.append(0)
    rom_values.append(1)

for i in range(16):
    rom_values.append(1)

rom_values.append(0)
# End of array creation

# Cast list to numpy array
rom_values = np.array(rom_values)
# Cast all values to integers
rom_values = np.uint16(rom_values)

# Write the ROM
result = write_rom(file_name, entity_name, n_addr, rom_values)

############## ENABLE RF 2 ##############
# Final VHDL file name
file_name = "stimulus_list_clk_enable_rf_2" 

# ROM Entity name
entity_name = file_name

# ROM Entity name
n_addr = 94

# Creating the array (using Antonio suggestion)
rom_values = []
rom_values.append(0)

for i in range(76):
    rom_values.append(1)

for i in range(4):
    rom_values.append(0)
    rom_values.append(0)
    rom_values.append(0)
    rom_values.append(1)

rom_values.append(0)
# End of array creation

# Cast list to numpy array
rom_values = np.array(rom_values)
# Cast all values to integers
rom_values = np.uint16(rom_values)

# Write the ROM
result = write_rom(file_name, entity_name, n_addr, rom_values)


C:\Users\Antonio\AppData\Local\Temp\ipykernel_5996\1889280955.py:32: UserWarning: The number of addresses it not an exponent of 2. You may have addresses with value output to zero.
  warnings.warn("The number of addresses it not an exponent of 2. You may have addresses with value output to zero.")
